<a href="https://colab.research.google.com/github/SravaniYalaganamoni/NEON/blob/main/c226f25_DBwithNEONandColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install SQLAlchemy psycopg2-binary pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 27.4 MB/s eta 0:00:00


In [2]:
# Simplified query cell (uses session-level search_path)
# - Connect with the STUDENT URL
# - Set search_path so we can omit schema prefixes
# - Run example queries and show results

import os, getpass, pandas as pd
from sqlalchemy import create_engine, text

STUDENT_DB_URL = os.environ.get("NEON_STUDENT_DB_URL") or getpass.getpass("Paste STUDENT DB URL (hidden): ")
engine = create_engine(STUDENT_DB_URL, pool_pre_ping=True, future=True)

# Example 1: quick row counts
with engine.connect() as conn:
    counts_sql = """
    SELECT 'product' AS table, COUNT(*) AS rows FROM product
    UNION ALL SELECT 'pc', COUNT(*) FROM pc
    UNION ALL SELECT 'laptop', COUNT(*) FROM laptop
    UNION ALL SELECT 'printer', COUNT(*) FROM printer;
    """
    counts_df = pd.read_sql_query(text(counts_sql), conn)

# Example 2: laptops with screens > 15" (maker, model, screen, price)
with engine.connect() as conn:
    laptops_sql = """
    SELECT p.maker, l.model, l.screen, l.price
    FROM product p
    JOIN laptop l USING (model)
    WHERE l.screen > 15
    ORDER BY l.screen DESC, l.price DESC;
    """
    laptops_df = pd.read_sql_query(text(laptops_sql), conn)

# Display results
print("Row counts:")
display(counts_df)

print("\nLaptops (screen > 15\"):")
display(laptops_df)


Paste STUDENT DB URL (hidden): ··········
Row counts:


,table,rows
0,product,30
1,pc,13
2,laptop,10
3,printer,7



Laptops (screen > 15"):


,maker,model,screen,price
0,E,2001,20.1,3673
1,A,2005,17.0,2500
2,E,2002,17.0,949
3,E,2003,16.4,549
4,G,2010,15.4,2300
5,A,2006,15.4,1700
6,F,2008,15.4,900


In [8]:
import os, getpass, pandas as pd
from sqlalchemy import create_engine, text

STUDENT_DB_URL = os.environ.get("NEON_STUDENT_DB_URL") or getpass.getpass("Paste STUDENT DB URL (hidden): ")
engine = create_engine(STUDENT_DB_URL, pool_pre_ping=True, future=True)

def run_query(sql: str) -> pd.DataFrame:
    with engine.connect() as conn:
        df = pd.read_sql_query(text(sql), conn)
    return df

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 20)
print("Engine ready. ✅")



Paste STUDENT DB URL (hidden): ··········
Engine ready. ✅


In [9]:
# Sanity check
counts_sql = '''
SELECT 'product' AS table, COUNT(*) AS rows FROM product
UNION ALL SELECT 'pc', COUNT(*) FROM pc
UNION ALL SELECT 'laptop', COUNT(*) FROM laptop
UNION ALL SELECT 'printer', COUNT(*) FROM printer;
'''
run_query(counts_sql)

,table,rows
0,product,30
1,pc,13
2,laptop,10
3,printer,7


In [10]:
# Query 1
q1 = '''
WITH catalog AS (
  SELECT p.maker, p.model, p.type,
         COALESCE(pc.price, l.price, pr.price) AS price
  FROM product p
  LEFT JOIN pc      pc  ON pc.model  = p.model
  LEFT JOIN laptop  l   ON l.model   = p.model
  LEFT JOIN printer pr  ON pr.model  = p.model
  WHERE COALESCE(pc.price, l.price, pr.price) IS NOT NULL
),
ranked AS (
  SELECT maker, model, type, price,
         DENSE_RANK() OVER (PARTITION BY type ORDER BY price DESC) AS rnk
  FROM catalog
)
SELECT type, maker, model, price
FROM ranked
WHERE rnk <= 3
ORDER BY type, price DESC, maker, model;
'''
run_query(q1)

,type,maker,model,price
0,laptop,E,2001,3673
1,laptop,A,2005,2500
2,laptop,G,2010,2300
3,pc,A,1001,2114
4,pc,B,1006,1049
5,pc,A,1002,995
6,printer,E,3003,899
7,printer,E,3002,239
8,printer,H,3007,200


In [13]:
# Query 2
q2 = '''
SELECT maker
FROM product p
GROUP BY maker
HAVING
  SUM(CASE WHEN type = 'pc' THEN 1 ELSE 0 END) > 0
  AND SUM(CASE WHEN type = 'laptop' THEN 1 ELSE 0 END) > 0
  AND SUM(CASE WHEN type = 'printer' THEN 1 ELSE 0 END) > 0
ORDER BY maker;
'''
run_query(q2)


,maker
0,E


In [3]:
# --- FULL WORKING SCRIPT (all-in-one) ---

import os, getpass, pandas as pd
from sqlalchemy import create_engine, text

# Connect to Neon (Postgres) student DB
STUDENT_DB_URL = os.environ.get("NEON_STUDENT_DB_URL") or getpass.getpass("Paste STUDENT DB URL (hidden): ")
engine = create_engine(STUDENT_DB_URL, pool_pre_ping=True, future=True)

# Define helper so it's always available
def run_query(sql: str) -> pd.DataFrame:
    with engine.connect() as conn:
        df = pd.read_sql_query(text(sql), conn)
    return df

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 20)
print("Engine ready. ✅")

# --- Query 3 directly (Laptop Value Score) ---
q3 = '''
SELECT p.maker,
       l.model,
       l.speed,
       l.ram,
       l.hd,
       l.screen,
       l.price,
       ROUND(
         (l.price::numeric /
          NULLIF((l.speed::numeric * (l.ram::numeric / 1024)), 0)
         ), 2
       ) AS value_score
FROM product p
JOIN laptop l USING (model)
WHERE l.speed IS NOT NULL
  AND l.ram IS NOT NULL
  AND l.price IS NOT NULL
ORDER BY value_score ASC, l.price ASC
LIMIT 15;
'''

print("\nQuery 3: Laptop Value Score (Price vs Speed & RAM)")
display(run_query(q3))


Paste STUDENT DB URL (hidden): ··········
Engine ready. ✅

Query 3: Laptop Value Score (Price vs Speed & RAM)


,maker,model,speed,ram,hd,screen,price,value_score
0,A,2006,2.00,2048,80,15.4,1700,425.00
1,E,2002,1.73,1024,80,17.0,949,548.55
2,F,2008,1.60,1024,100,15.4,900,562.50
3,G,2010,2.00,2048,160,15.4,2300,575.00
4,E,2003,1.80,512,60,16.4,549,610.00
5,B,2007,1.83,1024,120,13.3,1429,780.87
6,F,2009,1.60,512,80,14.1,680,850.00
7,E,2001,2.00,2048,240,20.1,3673,918.25
8,A,2004,2.00,512,60,13.3,1150,1150.00
9,A,2005,2.16,1024,120,17.0,2500,1157.41
